# Preliminary API

In [1]:
import TWB
import glob
import json

## Load data

In [2]:
# Make sure that the tags are also wikipedia articles!!!

with open('data/tags.json') as ifd:
    tags = json.load(ifd)
#ewith

api_key = '34e73062364d41b180a6f6a9625bff79'

## Load the API

In [3]:
data_location = '/Users/thiesgehrmann/Downloads/TWB/documents/hackathon-for-good-2019_TWB-challenge_files/*.sdlxliff'
xliff_files = glob.glob(data_location)
xliff = [ TWB.XLIFF(f) for f in xliff_files[:100]]

In [4]:
api = TWB.API(tags, api_key)

## Get data for xliffs from the API

In [5]:
X = api.get(xliff[:20])

In [3]:
topics = ['disability', 
#'environmental crisis', 
'gender', 
'genital mutilation', 
'humanitarian crisis', 
'natural disaster', 
'racism']
for topic in topics:
    trend = TWB.Trends(None, topic=topic)
    print(topic)
    trend.preload()

disability
gender
genital mutilation
humanitarian crisis
natural disaster
racism


In [10]:
trend.preload()

,ABW,AFG,ARE,ARG,ARM,ATG,AUS,AZE,BDI,BEL,...,TCD,THA,TUN,TUR,TUV,TZA,UGA,ZAF,ZMB,ZWE
date,,,,,,,,,,,,,,,,,,,,,
2004-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
pd.to_pickle

ABW     0.0
DOM     0.0
ECU     0.0
MWI     0.0
EST     0.0
FIN     0.0
SDN     0.0
PSE     0.0
PRY     0.0
GIB     0.0
GMB     0.0
HRV     0.0
DMA     0.0
HUN     0.0
IRN     0.0
ZMB     0.0
PCN     0.0
OMN     0.0
NRU     0.0
KAZ     0.0
NER     0.0
NCL     0.0
LVA     0.0
MYT     0.0
MLI     0.0
IMN     0.0
SOM     0.0
SLE     0.0
MNP     0.0
BHR     0.0
       ... 
MLT     7.0
NGA     8.0
NOR     8.0
BGD     8.0
ISR     9.0
THA     9.0
BRA     9.0
PAN     9.0
LBN     9.0
CHN     9.0
PRI    10.0
TUR    10.0
NZL    10.0
UGA    10.0
ITA    10.0
GHA    11.0
PAK    12.0
JOR    12.0
MYS    14.0
MEX    18.0
KEN    21.0
ZAF    22.0
JEY    25.0
SGP    27.0
PHL    28.0
IRL    39.0
IND    39.0
AUS    40.0
GEO    42.0
CAN    45.0
Length: 109, dtype: float64

In [13]:
pd.read_pickle('./data/trends/topics/disability/JORDAN.pkl')

,JORDAN disability,KAZAKHSTAN disability,isPartial
date,,,
2004-01-01,0,0,False
2004-02-01,81,0,False
2004-03-01,100,0,False
2004-04-01,0,0,False
2004-05-01,0,0,False
2004-06-01,0,0,False
2004-07-01,0,0,False
2004-08-01,0,0,False
2004-09-01,60,0,False


In [7]:
pd.__version__

NameError: name 'pd' is not defined

In [6]:
X

[{'target_lsc': code(language=language(iso3='BEN', iso2='BN', english='BENGALI', french='BENGALI', german='BENGALI'), script=script(code='LATN', number=215, name='LATIN', alias='LATIN', direction='L-TO-R', version=1.0, characters=1366.0, remarks='SEE LATIN SCRIPT IN UNICODE'), country=country(country='INDIA', iso2='IN', iso3='IND', un=356)),
  'source_lsc': code(language=language(iso3='ENG', iso2='EN', english='ENGLISH', french='ANGLAIS', german='ENGLISCH'), script=script(code='LATN', number=215, name='LATIN', alias='LATIN', direction='L-TO-R', version=1.0, characters=1366.0, remarks='SEE LATIN SCRIPT IN UNICODE'), country=country(country='UNITED STATES OF AMERICA', iso2='US', iso3='USA', un=840)),
  'topics': {'children': {'distance': 2.0, 'relevant_country_languages': {}},
   'ebola': {'distance': 2.449489742783178, 'relevant_country_languages': {}},
   'aids': {'distance': 2.449489742783178, 'relevant_country_languages': {}},
   'hiv': {'distance': 2.449489742783178, 'relevant_count